# Update Datasets

Outline:

- PreReq
- Ticker List
- Static Data:
-- Quotes
-- Description
-- Fundamental
- Live Data:
-- Intraday Quotes
-- News

## Pre-Requisites

### G-Drive Connect
Authorise google colaboratory to access drive.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Requirements

In [2]:
!pip install yfinance finvizfinance finviz finpie yahoo_fin

     |████████████████████████████████| 13.5MB 250kB/s 
     |████████████████████████████████| 5.5MB 46.9MB/s 
     |████████████████████████████████| 61kB 5.9MB/s 
     |████████████████████████████████| 1.3MB 45.4MB/s 
     |████████████████████████████████| 911kB 43.0MB/s 
     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 245kB 42.0MB/s 
     |████████████████████████████████| 143kB 52.5MB/s 
     |████████████████████████████████| 296kB 41.6MB/s 
     |████████████████████████████████| 112kB 36.6MB/s 
     |████████████████████████████████| 675kB 41.4MB/s 
     |████████████████████████████████| 92kB 7.3MB/s 
     |████████████████████████████████| 81kB 7.5MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=0d77c03652780554eaaff2e9fdfc0b1cec11d1a12f6a91f84a777a91e4870499
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d6

### Import dependencies

In [3]:
import pandas as pd
import finpie
import yfinance as yf
import finviz
import finvizfinance
import time
import progressbar
from finvizfinance.screener.overview import Overview
from yahoo_fin import stock_info as si

## Static Data

The static data records the historic price values for the equities. This should be updated after the markets close at 8 pm EST.

### Ticker List
The ticker symbol list needs to be updated as new symbols are being listed/delisted everyday.

In [4]:
def update_ticker_list():
  # Download latest ticker list from Nasdaq
  ticker_list=finpie.nasdaq_tickers()
  # Select Symbol Column
  ticker_list=ticker_list['Symbol']
  # Save ticker list to csv
  ticker_list.to_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/symbols.csv',index=None)
  # Download latest ticker list in S&P500
  foverview = Overview()
  filters_dict = {'Index':'S&P 500','Sector':'Any'}
  foverview.set_filter(filters_dict=filters_dict)
  df = foverview.ScreenerView()
  # Select Symbol Column
  df['Symbol']=df['Ticker']
  df=df['Symbol']
  # Save ticker list to csv
  df.to_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/S&P500.csv',index=None)
  print("Success")

### Stock Data

The download functionality takes about 3 hours to completely download historic data for all 9000+ symbols. Modify your serach criteria for downloading datasets to save time.

In [5]:
def update_historical_price(letter="sp500"):
  symbols=pd.read_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/symbols.csv')
  if letter == "all":
    symbols=pd.read_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/symbols.csv')
  elif letter == "sp500":
    symbols=pd.read_csv('/content/drive/My Drive/Master Project/Dataset/stocks_ticker_list_latest/S&P500.csv')
  else:
    symbols=symbols[[x.startswith(letter) for x in symbols['Symbol']]]

  bar = progressbar.ProgressBar(
      widgets=[
          progressbar.Counter(format='%(value)5d/%(max_value)d | %(dynamic_messages)6s | '),
          progressbar.Timer(format= 'Elapsed: %(elapsed)s '),
          progressbar.Bar('*'),
          ' ',
          progressbar.AdaptiveETA(),
      ],
  )

  for symbol in bar(symbols['Symbol']):
    try:
      bar.dynamic_messages=symbol
      data=finpie.historical_prices(symbol)
      data.to_csv('/content/drive/My Drive/Master Project/Dataset/stock_market_datasets/market_historic/{}.csv'.format(symbol))
      #print(symbol)
      #time.sleep(1)
    except:
      pass

In [7]:
finpie.historical_prices("PLTR")

,open,high,low,close,adj_close,volume
date,,,,,,
2020-09-30,10.000000,11.410000,9.110000,9.500000,9.500000,338584400
2020-10-01,9.690000,10.100000,9.230000,9.460000,9.460000,124297600
2020-10-02,9.060000,9.280000,8.940000,9.200000,9.200000,55018300
2020-10-05,9.430000,9.490000,8.920000,9.030000,9.030000,36316900
2020-10-06,9.040000,10.180000,8.900000,9.900000,9.900000,90864000
...,...,...,...,...,...,...
2021-02-22,29.959999,30.190001,27.535999,28.000000,28.000000,181564500
2021-02-23,25.959999,27.459999,23.900000,26.750000,26.750000,144516900
2021-02-24,26.709999,27.230000,25.750000,26.389999,26.389999,89320100


In [8]:
si.get_data("PLTR")

,open,high,low,close,adjclose,volume,ticker
2020-09-30,10.000000,11.410000,9.110000,9.500000,9.500000,338584400,PLTR
2020-10-01,9.690000,10.100000,9.230000,9.460000,9.460000,124297600,PLTR
2020-10-02,9.060000,9.280000,8.940000,9.200000,9.200000,55018300,PLTR
2020-10-05,9.430000,9.490000,8.920000,9.030000,9.030000,36316900,PLTR
2020-10-06,9.040000,10.180000,8.900000,9.900000,9.900000,90864000,PLTR
...,...,...,...,...,...,...,...
2021-02-23,25.959999,27.459999,23.900000,26.750000,26.750000,144516900,PLTR
2021-02-24,26.709999,27.230000,25.750000,26.389999,26.389999,89320100,PLTR
2021-02-25,25.820000,26.440001,23.150000,23.959999,23.959999,160304800,PLTR
2021-02-26,24.200001,24.500000,22.700001,23.900000,23.900000,118368700,PLTR


In [9]:
msft = yf.Ticker("MSFT")
# get stock info
msft.info
# get historical market data
hist = msft.history(period="max")
# show actions (dividends, splits)
msft.actions
# show dividends
msft.dividends
# show splits
msft.splits
# show financials
msft.financials
msft.quarterly_financials
# show major holders
msft.major_holders
# show institutional holders
msft.institutional_holders
# show balance sheet
msft.balance_sheet
msft.quarterly_balance_sheet
# show cashflow
msft.cashflow
msft.quarterly_cashflow
# show earnings
msft.earnings
msft.quarterly_earnings
# show sustainability
msft.sustainability
# show analysts recommendations
msft.recommendations
# show next event (earnings, etc)
msft.calendar
# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
#msft.isin
# show options expirations
#options_dates=msft.options
# get option chain for specific expiration
#opt = msft.option_chain('2021-03-19')
# data available via: opt.calls, opt.puts
#opt.calls

,0,1
Earnings Date,2021-04-27 00:00:00,2021-05-03 00:00:00
Earnings Average,1.77,1.77
Earnings Low,1.69,1.69
Earnings High,1.93,1.93
Revenue Average,41032800000,41032800000
Revenue Low,40541000000,40541000000
Revenue High,41894000000,41894000000


In [10]:
calls,puts=finpie.yahoo_option_chain("MSFT")

In [11]:
#finpie.historical_futures_contracts( pd.date_range('2020-01-01', '2020-03-01') )

100%|██████████| 61/61 [01:06<00:00,  1.09s/it]


,month,date,open,high,low,close,change,volume,open_interest,change_in_oi,future
date,,,,,,,,,,,
2020-01-01,Jan20,191231.0,298.10,302.00,296.80,299.90,2.40,41843.0,19917.0,-12579.0,Soybean E6eal(CE9OE12)
2020-01-01,E6ar20,191231.0,302.80,307.10,301.70,304.70,2.40,64699.0,213435.0,5357.0,Soybean E6eal(CE9OE12)
2020-01-01,E6ay20,191231.0,306.20,310.70,305.70,308.50,2.40,13703.0,90147.0,86.0,Soybean E6eal(CE9OE12)
2020-01-01,Jul20,191231.0,309.90,314.30,309.30,312.00,2.20,8121.0,54704.0,206.0,Soybean E6eal(CE9OE12)
2020-01-01,Aug20,191231.0,311.50,315.60,310.70,313.20,2.10,1546.0,10822.0,31.0,Soybean E6eal(CE9OE12)
...,...,...,...,...,...,...,...,...,...,...,...
2020-02-28,Jun20,200228.0,21430.00,21720.00,21150.00,21434.00,-782.00,237.0,3379.0,146.0,FE12SE E6IE9(ISE)
2020-02-28,Sep20,200228.0,21500.00,21500.00,21277.00,21277.00,-784.00,0.0,47.0,0.0,FE12SE E6IE9(ISE)
2020-02-28,E6ar20,200228.0,273.55,274.15,267.55,267.85,-9.90,487435.0,326122.0,8019.0,E3OSPI 200(E3FE)


## Intraday (1 month)

# Main

In [12]:
update_ticker_list()

Success


In [13]:
update_historical_price('PLTR')

    1/1 |   PLTR | Elapsed: 0:00:56 |**************************| Time:  0:00:56


# References

https://github.com/ranaroussi/yfinance/tree/master/yfinance

https://github.com/peterlacour/finpie

https://github.com/mariostoev/finviz

https://github.com/lit26/finvizfinance

## News

In [14]:
!pip install stocknews

  Created wheel for stocknews: filename=stocknews-0.9.11-cp37-none-any.whl size=5413 sha256=7701fe7a71ec742ff62b25c517b8f930c7a431feed507c90275caaa67426da3a
  Stored in directory: /root/.cache/pip/wheels/99/17/8f/a6f14b6c34ba888067879d6acf067f8f63238223d2703a91dd
Successfully built stocknews


In [15]:
from stocknews import StockNews

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
